In [2]:
import pickle
import jax
import jax.numpy as jnp
from dataclasses import asdict
import jax.tree_util as tree
import numpy as np
import torch.onnx

In [3]:
with open("utils/weights/go2_params.pkl", "rb") as f:
    params_loaded = pickle.load(f)

# Convert back to JAX arrays if needed
params_jax = jax.tree_map(jnp.array, params_loaded)

print("Params successfully loaded")

/tmp/ipykernel_39242/3522588695.py:5: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  params_jax = jax.tree_map(jnp.array, params_loaded)


Params successfully loaded


In [4]:
mean_std_all = asdict(params_loaded[0])

In [5]:
mean_std_all

{'mean': {'privileged_state': array([-5.37527027e-03, -3.72266746e-03,  8.12138896e-03,  2.57836631e-03,
         -6.08543726e-03, -1.65748596e-03,  6.04978902e-03, -1.80347846e-03,
         -9.96224463e-01,  9.29868402e-05,  2.46114898e-02,  1.09104462e-01,
          1.61302444e-02,  1.28113348e-02,  9.11557004e-02, -1.92847494e-02,
          9.35180578e-03,  1.01881891e-01,  3.81241962e-02,  2.00909637e-02,
          8.42443928e-02, -4.69057495e-03,  8.90833884e-03, -2.58548930e-02,
          5.94130391e-03,  9.43633262e-03, -2.62726732e-02, -6.10564649e-03,
          4.42138221e-03, -2.11185254e-02,  6.94550341e-03, -1.06999592e-03,
         -6.93152659e-03, -2.30879664e-01, -3.03159729e-02,  5.01185060e-01,
          2.54054993e-01, -1.06226727e-02,  5.21934569e-01, -2.14919150e-01,
          5.40529490e-02,  4.64715093e-01,  2.54152119e-01,  3.86505276e-02,
          4.87904727e-01, -2.90122745e-03, -7.52538326e-04, -1.18151722e-04,
          2.57487851e-03, -6.08745031e-03, -1.66

In [6]:
weights_bias = params_loaded[1]

In [7]:
weights_bias["params"].keys()

dict_keys(['hidden_0', 'hidden_1', 'hidden_2', 'hidden_3'])

In [8]:
weights_bias["params"]['hidden_0'].keys()

dict_keys(['bias', 'kernel'])

In [9]:
weights_bias["params"]['hidden_0']['bias'].shape

(512,)

In [10]:
weights_bias["params"]['hidden_0']['kernel'].shape

(48, 512)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MLP(nn.Module):
    def __init__(
        self,
        layer_sizes,
        activation=nn.ReLU(),
        kernel_init="lecun_uniform",
        activate_final=False,
        bias=True,
        layer_norm=False,
        mean_std=None,
    ):
        super().__init__()

        self.layer_sizes = layer_sizes
        self.activation = activation
        self.kernel_init = kernel_init
        self.activate_final = activate_final
        self.bias = bias
        self.layer_norm = layer_norm

        # Register mean and std as buffers (non-trainable parameters)
        if mean_std is not None:
            self.register_buffer("mean", torch.tensor(mean_std[0], dtype=torch.float32))
            self.register_buffer("std", torch.tensor(mean_std[1], dtype=torch.float32))
        else:
            self.mean = None
            self.std = None

        # Build the MLP block
        self.mlp_block = nn.Sequential()
        for i in range(len(self.layer_sizes) - 1):
            in_features = self.layer_sizes[i]
            out_features = self.layer_sizes[i + 1]

            # Add linear layer
            dense_layer = nn.Linear(in_features, out_features, bias=self.bias)
            self.mlp_block.add_module(f"hidden_{i}", dense_layer)

            # Initialize weights (e.g., Lecun uniform initialization)
            if self.kernel_init == "lecun_uniform":
                nn.init.kaiming_uniform_(dense_layer.weight, mode='fan_in', nonlinearity='relu')

            # Add layer normalization if enabled
            if self.layer_norm and i < len(self.layer_sizes) - 2:  # No layer norm after the last layer
                self.mlp_block.add_module(f"layer_norm_{i}", nn.LayerNorm(out_features))

            # Add activation function, except for the final layer if `activate_final` is False
            if i < len(self.layer_sizes) - 2 or self.activate_final:  # Add activation for all but the last layer
                self.mlp_block.add_module(f"activation_{i}", self.activation)

    def forward(self, inputs):
        # Handle list inputs
        if isinstance(inputs, list):
            inputs = inputs[0]

        # Normalize inputs if mean and std are provided
        if self.mean is not None and self.std is not None:
            inputs = (inputs - self.mean) / self.std

        # Pass through the MLP block
        logits = self.mlp_block(inputs)

        # Split the output into two parts and apply tanh to the first half
        loc, _ = torch.split(logits, logits.size(-1) // 2, dim=-1)
        return torch.tanh(loc)

def make_policy_network(
    observation_size,
    action_size,
    mean_std,
    hidden_layer_sizes=[256, 256],
    activation=nn.ReLU(),
    kernel_init="lecun_uniform",
    layer_norm=False,
):
    layers = hidden_layer_sizes + [action_size]
    print(layers)
    policy_network = MLP(
        layer_sizes= layers,
        activation=activation,
        kernel_init=kernel_init,
        layer_norm=layer_norm,
        mean_std=mean_std,
    )
    return policy_network

In [12]:
mean_std = (torch.tensor(mean_std_all['mean']['state']), torch.tensor(mean_std_all['std']['state']))

In [13]:
th_policy_network = make_policy_network(
    observation_size = 48,
    action_size = 12*2,
    mean_std=mean_std,
    hidden_layer_sizes=[48,512, 256, 128])

[48, 512, 256, 128, 24]


/tmp/ipykernel_39242/1165096672.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("mean", torch.tensor(mean_std[0], dtype=torch.float32))
/tmp/ipykernel_39242/1165096672.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("std", torch.tensor(mean_std[1], dtype=torch.float32))


In [14]:
th_policy_network

MLP(
  (activation): SiLU()
  (mlp_block): Sequential(
    (hidden_0): Linear(in_features=48, out_features=512, bias=True)
    (activation_0): SiLU()
    (hidden_1): Linear(in_features=512, out_features=256, bias=True)
    (activation_1): SiLU()
    (hidden_2): Linear(in_features=256, out_features=128, bias=True)
    (activation_2): SiLU()
    (hidden_3): Linear(in_features=128, out_features=24, bias=True)
  )
)

In [15]:
# Access and modify weights and biases
for i, layer in enumerate(th_policy_network.mlp_block):
    if isinstance(layer, nn.Linear):
        print(f"Layer {i}:")
        print("Original weights:", layer.weight.shape)
        print("Original biases:", layer.bias.shape)

Layer 0:
Original weights: torch.Size([512, 48])
Original biases: torch.Size([512])
Layer 2:
Original weights: torch.Size([256, 512])
Original biases: torch.Size([256])
Layer 4:
Original weights: torch.Size([128, 256])
Original biases: torch.Size([128])
Layer 6:
Original weights: torch.Size([24, 128])
Original biases: torch.Size([24])


In [16]:
type(weights_bias["params"]['hidden_0'])

dict

In [17]:
for key, values in weights_bias["params"].items():
    print(f"layer:{key}------------------")
    for key, kernel_bias in values.items():
        print(f"{key} size: {kernel_bias.shape}")

layer:hidden_0------------------
bias size: (512,)
kernel size: (48, 512)
layer:hidden_1------------------
bias size: (256,)
kernel size: (512, 256)
layer:hidden_2------------------
bias size: (128,)
kernel size: (256, 128)
layer:hidden_3------------------
bias size: (24,)
kernel size: (128, 24)


In [18]:
for i, layer in enumerate(th_policy_network.mlp_block):
    print(i, layer)

0 Linear(in_features=48, out_features=512, bias=True)
1 SiLU()
2 Linear(in_features=512, out_features=256, bias=True)
3 SiLU()
4 Linear(in_features=256, out_features=128, bias=True)
5 SiLU()
6 Linear(in_features=128, out_features=24, bias=True)


In [18]:
# Assuming th_policy_network is already defined
values = [(key,value) for key, value in weights_bias["params"].items()]
j = 0
for i, layer in enumerate(th_policy_network.mlp_block):
    if isinstance(layer, nn.Linear):  # Check if the layer is a Linear layer
        #print(f"Layer {values[i][0]}:")
        #print("Weights shape:", layer.weight.shape)
        #print("Biases shape:", layer.bias.shape)
        #print(f"before: {layer.weight.data}, size: {layer.weight.data.shape}")
        #print(f"BEFOREvalues: {values[i][1]['kernel']}, size: {values[i][1]['kernel'].shape}")

        #print(f"size: {values[i][1]['bias'].shape}")
        #print(f"size: {values[i][1]['kernel'].shape}")
        if (i%2==0):
            
            transpose_tensor_kernel = torch.tensor(values[j][1]['kernel']).t()
            transpose_tensor_bias = torch.tensor(values[j][1]['bias']).t()

            #print(f"ker: {transpose_tensor_kernel.shape}")
            #print(f"bia: {transpose_tensor_bias.shape}")
                # Assign new weights and biases
            layer.weight.data = transpose_tensor_kernel
            layer.bias.data = transpose_tensor_bias
            j = j + 1
     

In [19]:
batch_size = 1
input = torch.randn(batch_size, 48)  # For a batch of inputs

In [20]:
th_policy_network

MLP(
  (activation): SiLU()
  (mlp_block): Sequential(
    (hidden_0): Linear(in_features=48, out_features=512, bias=True)
    (activation_0): SiLU()
    (hidden_1): Linear(in_features=512, out_features=256, bias=True)
    (activation_1): SiLU()
    (hidden_2): Linear(in_features=256, out_features=128, bias=True)
    (activation_2): SiLU()
    (hidden_3): Linear(in_features=128, out_features=24, bias=True)
  )
)

In [21]:
input.shape

torch.Size([1, 48])

In [22]:
th_policy_network.forward(input)

tensor([[-0.1688,  0.8599,  0.9902,  0.1623, -0.0766, -0.3626,  0.7645,  0.9771,
          0.9611,  0.8755, -0.2907, -0.5002]], grad_fn=<TanhBackward0>)

In [23]:
mean_std

(tensor([-5.3753e-03, -3.7227e-03,  8.1214e-03,  2.5784e-03, -6.0854e-03,
         -1.6575e-03,  6.0498e-03, -1.8035e-03, -9.9622e-01,  9.2987e-05,
          2.4611e-02,  1.0910e-01,  1.6130e-02,  1.2811e-02,  9.1156e-02,
         -1.9285e-02,  9.3518e-03,  1.0188e-01,  3.8124e-02,  2.0091e-02,
          8.4244e-02, -4.6906e-03,  8.9083e-03, -2.5855e-02,  5.9413e-03,
          9.4363e-03, -2.6273e-02, -6.1056e-03,  4.4214e-03, -2.1119e-02,
          6.9455e-03, -1.0700e-03, -6.9315e-03, -2.3088e-01, -3.0316e-02,
          5.0119e-01,  2.5405e-01, -1.0623e-02,  5.2193e-01, -2.1492e-01,
          5.4053e-02,  4.6472e-01,  2.5415e-01,  3.8651e-02,  4.8790e-01,
         -2.9012e-03, -7.5254e-04, -1.1815e-04]),
 tensor([0.3682, 0.1376, 0.1472, 0.8580, 0.4785, 0.5599, 0.0573, 0.0689, 0.0445,
         0.1033, 0.1173, 0.1269, 0.1088, 0.1202, 0.1283, 0.0904, 0.1269, 0.1364,
         0.0931, 0.1213, 0.1330, 1.9139, 2.7987, 3.6095, 1.9457, 2.8429, 3.8450,
         1.9158, 3.0908, 3.5546, 1.9811, 

In [24]:

# Define the output ONNX file path
onnx_file_path = "sai.onnx"

# Export the model
torch.onnx.export(
    th_policy_network,                  # Model to export
    input,            # Dummy input
    onnx_file_path,         # Output file path
    export_params=True,     # Export model parameters (weights)
    opset_version=11,       # ONNX opset version (e.g., 11 is widely supported)
    do_constant_folding=True,  # Optimize the model by folding constants
    input_names=["state"],  # Input tensor name
    output_names=["actions"],  # Output tensor name
)

print(f"Model exported to {onnx_file_path}")

Model exported to sai.onnx
